## Solve an inverse tarffic problem over polynomials of degree at most d

## Optionally use a regularizer from the poly kernel

In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import json

In [3]:
# read in node-link incidence matrtix
with open('node_link_incidence_Sioux.json', 'r') as json_file:
    N_dict = json.load(json_file)
    
N = zload('node_link_incidence_Sioux.pkz')

In [4]:
N_dict

{u'4-34': 0.0,
 u'4-35': 0.0,
 u'4-36': 0.0,
 u'4-37': 0.0,
 u'4-30': 0.0,
 u'4-31': 0.0,
 u'4-32': 0.0,
 u'4-33': 0.0,
 u'4-38': 0.0,
 u'4-39': 0.0,
 u'10-49': 0.0,
 u'10-48': 0.0,
 u'10-45': 0.0,
 u'10-44': 0.0,
 u'10-47': 0.0,
 u'10-46': 0.0,
 u'10-41': 0.0,
 u'10-40': 0.0,
 u'10-43': 0.0,
 u'10-42': 0.0,
 u'21-8': 0.0,
 u'21-9': 0.0,
 u'21-4': 0.0,
 u'21-5': 0.0,
 u'21-6': 0.0,
 u'21-7': 0.0,
 u'21-0': 0.0,
 u'21-1': 0.0,
 u'21-2': 0.0,
 u'21-3': 0.0,
 u'17-26': 0.0,
 u'17-27': 0.0,
 u'17-24': 0.0,
 u'17-25': 0.0,
 u'17-22': 0.0,
 u'17-23': 0.0,
 u'17-20': 0.0,
 u'17-21': 0.0,
 u'12-47': 0.0,
 u'12-46': 0.0,
 u'12-45': 0.0,
 u'12-44': 0.0,
 u'12-43': 0.0,
 u'12-42': 0.0,
 u'12-41': 0.0,
 u'12-40': 0.0,
 u'3-75': 0.0,
 u'3-74': 0.0,
 u'3-71': 0.0,
 u'3-70': 0.0,
 u'3-73': 0.0,
 u'3-72': 0.0,
 u'6-54': 0.0,
 u'6-55': 0.0,
 u'6-56': 0.0,
 u'6-57': 0.0,
 u'6-50': 0.0,
 u'6-51': 0.0,
 u'6-52': 0.0,
 u'6-53': -1.0,
 u'11-60': 0.0,
 u'11-61': 0.0,
 u'11-62': 0.0,
 u'11-63': 0.0,
 u'11-64'

In [5]:
N.shape

(24, 76)

In [6]:
# define polynomial kernel with degree d
def k(x, y, c, d):
    return (c + x*y) ** d

In [7]:
# construct kernel matrix; take d=5 for instance
c = 1.0
d = 5
pts = np.linspace(0.0, 1.0, num=d+1)

In [8]:
pts

array([ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ])

In [9]:
phiSimp_dict = {}
for i in range(d+1):
    for j in range(d+1):
        key = str(i) + ',' + str(j)
        phiSimp_dict[key] = k(pts[i], pts[j], c, d)

In [10]:
phiSimp_dict

{'0,0': 1.0,
 '0,1': 1.0,
 '0,2': 1.0,
 '0,3': 1.0,
 '0,4': 1.0,
 '0,5': 1.0,
 '1,0': 1.0,
 '1,1': 1.2166529024000001,
 '1,2': 1.4693280768000005,
 '1,3': 1.7623416832000007,
 '1,4': 2.1003416576000014,
 '1,5': 2.4883199999999994,
 '2,0': 1.0,
 '2,1': 1.4693280768000005,
 '2,2': 2.1003416576000014,
 '2,3': 2.9316250623999998,
 '2,4': 4.0074642432000012,
 '2,5': 5.3782399999999981,
 '3,0': 1.0,
 '3,1': 1.7623416832000007,
 '3,2': 2.9316250623999998,
 '3,3': 4.6525874176000013,
 '3,4': 7.1008211967999992,
 '3,5': 10.485760000000003,
 '4,0': 1.0,
 '4,1': 2.1003416576000014,
 '4,2': 4.0074642432000012,
 '4,3': 7.1008211967999992,
 '4,4': 11.863674982400005,
 '4,5': 18.895680000000002,
 '5,0': 1.0,
 '5,1': 2.4883199999999994,
 '5,2': 5.3782399999999981,
 '5,3': 10.485760000000003,
 '5,4': 18.895680000000002,
 '5,5': 32.0}

In [11]:
len(phiSimp_dict)

36